<a href="https://colab.research.google.com/github/cincysam6/Field_Control_Model/blob/main/interactive_parameter_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install shiny matplotlib numpy
# (and your package if needed)
# !pip install -e .

!shiny run --reload app.py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.2/452.2 kB 14.4 MB/s eta 0:00:00
Error: ./app.py not found



In [2]:
# 1. Clone your repo
!git clone https://github.com/cincysam6/Field_Control_Model.git

# 2. Add it to Python path
import sys
sys.path.append('/content/Field_Control_Model')

# 3. Import your helper function
from src.helpers import compute_distances_by_frame, pick_panel_kwargs,update_kwargs
from src.diagnostic_plots import plot_dir_orientation_small_multiples,diagnostic_plot,plot_field_control_small_multiples,diagnostic_multiples
from src.field_control_model import PlayerInfluenceModel
from src.compute_player_density import compute_player_densities_dataframe
from src.plot_player_density import plot_player_densities_from_dataframe, plot_team_densities_small_multiples
from src.animate_plays import animate_pitch_control_with_players, animate_pitch_control_with_players_fast
from src.compute_player_density import compute_player_densities_team_control
from src.plot_player_density import visualize_team_control
from src.presets import default_kwargs, triangular_kwargs

ModuleNotFoundError: No module named 'src'

In [ ]:
# === Launch a Shiny (Python) App in a new browser tab (Colab) ===
import random, threading, time
import shiny
from google.colab import output
from shiny import App, ui, render, reactive
from math import cos, sin
import numpy as np
import matplotlib.pyplot as plt
# === Shiny app for Colab: Field Control (opens in new tab + speed Play) ===
# pip installs (safe no-ops if already installed)
import sys, threading, time, random
import numpy as np
import matplotlib.pyplot as plt
from math import cos, sin
from shiny import App, ui, render, reactive


In [ ]:

# Try to import your real model from the current notebook session
try:
    from __main__ import PlayerInfluenceModel  # if class defined earlier in the notebook
    HAVE_MODEL = True
except Exception:
    HAVE_MODEL = False
    PlayerInfluenceModel = None

# ---------------- Presets ----------------
PRESETS = {
    "Balanced": dict(
        gaussian_scale_factor=0.68,
        alpha_gamma=6.0,
        beta_min=2.0, beta_max=18.0,
        gamma_midpoint=13.0,
        gamma_scale_factor=0.9,
        max_forward_distance=20.0,
        forward_decay_factor=1.1,
        angle_limit_min=15.0, angle_limit_max=55.0, angle_decay_factor=2.2,
        w_gaussian_min=0.2, w_gaussian_max=1.0,
        gaussian_midpoint=4.0, gaussian_steepness=1.8,
    ),
    "Conservative": dict(
        gaussian_scale_factor=0.7,
        alpha_gamma=11.0,
        beta_min=2.0, beta_max=15.0,
        gamma_midpoint=15.0,
        gamma_scale_factor=0.8,
        max_forward_distance=18.0,
        forward_decay_factor=1.0,
        angle_limit_min=20.0, angle_limit_max=50.0, angle_decay_factor=2.5,
        w_gaussian_min=0.3, w_gaussian_max=1.0,
        gaussian_midpoint=4.5, gaussian_steepness=2.0,
    ),
    "Aggressive wedge": dict(
        gaussian_scale_factor=0.65,
        alpha_gamma=3.5,
        beta_min=1.0, beta_max=25.0,
        gamma_midpoint=10.0,
        gamma_scale_factor=1.0,
        max_forward_distance=25.0,
        forward_decay_factor=1.3,
        angle_limit_min=12.0, angle_limit_max=60.0, angle_decay_factor=2.0,
        w_gaussian_min=0.15, w_gaussian_max=1.0,
        gaussian_midpoint=3.5, gaussian_steepness=1.5,
    ),
}

# ---------------- UI ----------------
sidebar = ui.sidebar(
    ui.input_select("preset", "Preset", list(PRESETS.keys()), selected="Balanced"),
    ui.input_action_button("apply", "Apply preset"),
    ui.input_action_button("reset", "Reset sliders"),

    ui.hr(),
    ui.input_text("name", "Name", "Adam Thielen"),
    ui.input_slider("speed", "Speed", min=0.0, max=11.5, value=5.0, step=0.1),
    ui.input_slider("dir_deg", "Direction°", min=0.0, max=360.0, value=0.0, step=1.0),
    ui.input_checkbox("show_o", "Show Orientation", False),
    ui.input_slider("o_deg", "Orient°", min=0.0, max=360.0, value=0.0, step=1.0),
    ui.input_slider("x", "X", min=40.0, max=80.0, value=60.0, step=0.5),
    ui.input_slider("y", "Y", min=10.0, max=43.3, value=26.65, step=0.5),
    ui.input_checkbox("zoom", "Zoom around X", False),
    ui.input_slider("zoom_pad_x", "Zoom pad", min=2.0, max=30.0, value=10.0, step=1.0),


    ui.hr(),
    ui.markdown("**Gaussian**"),
    ui.input_slider("gaussian_scale_factor", "Scale", min=0.4, max=1.2, value=0.68, step=0.02),
    ui.input_slider("low_speed_gaussian_cutoff", "Low-speed cutoff (yd/s)",  # NEW
                    min=0.0, max=5.0, value=2.0, step=0.5),


    ui.hr(),
    ui.markdown("**Gamma**"),
    ui.input_slider("alpha_gamma", "alpha", min=2.0, max=12.0, value=6.0, step=0.5),
    ui.input_slider("beta_min", "beta_min", min=0.5, max=8.0, value=2.0, step=0.5),
    ui.input_slider("beta_max", "beta_max", min=8.0, max=30.0, value=18.0, step=1.0),
    ui.input_slider("gamma_midpoint", "midpoint", min=6.0, max=20.0, value=13.0, step=0.5),
    ui.input_slider("gamma_scale_factor", "scale_fac", min=0.6, max=1.4, value=0.9, step=0.02),
    ui.input_slider("max_forward_distance", "cap", min=10.0, max=30.0, value=20.0, step=1.0),
    ui.input_slider("forward_decay_factor", "tail_decay", min=0.6, max=2.0, value=1.1, step=0.05),

    ui.hr(),
    ui.markdown("**Cone**"),
    ui.input_slider("angle_limit_min", "min° (fast)", min=8.0,  max=30.0, value=15.0, step=1.0),
    ui.input_slider("angle_limit_max", "max° (slow)", min=35.0, max=80.0, value=55.0, step=1.0),
    ui.input_slider("angle_decay_factor", "decay", min=1.0, max=4.0, value=2.2, step=0.1),

    ui.hr(),
    ui.markdown("**Weights**"),
    ui.input_slider("w_gaussian_min", "wG min", min=0.0, max=0.5, value=0.2, step=0.02),
    ui.input_slider("w_gaussian_max", "wG max", min=0.5, max=1.0, value=1.0, step=0.02),
    ui.input_slider("gaussian_midpoint", "midpoint", min=2.0, max=6.0, value=4.0, step=0.1),
    ui.input_slider("gaussian_steepness", "steepness", min=0.5, max=3.0, value=1.8, step=0.1),

    ui.hr(),
    ui.markdown("**Style**"),
    ui.input_slider("levels", "Levels", min=10, max=60, value=20, step=1),
    ui.input_select("cmap", "Colormap", ["OrRd","Reds","magma","inferno"], selected="OrRd"),
    ui.input_slider("fill_alpha", "Fill α", min=0.05, max=0.5, value=0.22, step=0.01),
    ui.input_slider("line_alpha", "Line α", min=0.1,  max=0.9, value=0.50, step=0.01),
    ui.input_slider("line_lw", "Line lw", min=0.2, max=2.0, value=0.6, step=0.1),

    ui.hr(),
    ui.markdown("**Play (speed sweep)**"),
    ui.input_checkbox("play", "Play", False),
    ui.input_slider("fps", "FPS", min=1, max=30, value=6, step=1),
    ui.input_slider("sweep_min", "Sweep min speed", min=0.0, max=11.5, value=0.0, step=0.1),
    ui.input_slider("sweep_max", "Sweep max speed", min=0.0, max=11.5, value=11.5, step=0.1),
    ui.input_slider("sweep_step", "Sweep step", min=0.05, max=1.0, value=0.1, step=0.05),
    open="open", width=370
)

app_ui = ui.page_sidebar(
    sidebar,
    ui.card(ui.output_plot("contour", height="700px"), full_screen=True),
    title="Field Control – Gaussian/Gamma"
)

# ---------------- Server ----------------
def server(input, output, session):
    # Apply preset
    @reactive.effect
    @reactive.event(input.apply)
    def _apply_preset():
        cfg = PRESETS[input.preset()]
        ui.update_slider(session, "gaussian_scale_factor", value=cfg["gaussian_scale_factor"])
        ui.update_slider(session, "alpha_gamma", value=cfg["alpha_gamma"])
        ui.update_slider(session, "beta_min", value=cfg["beta_min"])
        ui.update_slider(session, "beta_max", value=cfg["beta_max"])
        ui.update_slider(session, "gamma_midpoint", value=cfg["gamma_midpoint"])
        ui.update_slider(session, "gamma_scale_factor", value=cfg["gamma_scale_factor"])
        ui.update_slider(session, "max_forward_distance", value=cfg["max_forward_distance"])
        ui.update_slider(session, "forward_decay_factor", value=cfg["forward_decay_factor"])
        ui.update_slider(session, "angle_limit_min", value=cfg["angle_limit_min"])
        ui.update_slider(session, "angle_limit_max", value=cfg["angle_limit_max"])
        ui.update_slider(session, "angle_decay_factor", value=cfg["angle_decay_factor"])
        ui.update_slider(session, "w_gaussian_min", value=cfg["w_gaussian_min"])
        ui.update_slider(session, "w_gaussian_max", value=cfg["w_gaussian_max"])
        ui.update_slider(session, "gaussian_midpoint", value=cfg["gaussian_midpoint"])
        ui.update_slider(session, "gaussian_steepness", value=cfg["gaussian_steepness"])
        ui.update_slider(session, "low_speed_gaussian_cutoff", value=cfg["low_speed_gaussian_cutoff"])

    # Reset to Balanced
    @reactive.effect
    @reactive.event(input.reset)
    def _reset():
        ui.update_select(session, "preset", selected="Balanced")
        session.send_input_message("apply", {"value": (input.apply() or 0) + 1})

    def _model_kwargs():
        return dict(
            gaussian_scale_factor=input.gaussian_scale_factor(),
            alpha_gamma=input.alpha_gamma(),
            beta_min=input.beta_min(), beta_max=input.beta_max(),
            gamma_midpoint=input.gamma_midpoint(),
            gamma_scale_factor=input.gamma_scale_factor(),
            max_forward_distance=input.max_forward_distance(),
            forward_decay_factor=input.forward_decay_factor(),
            angle_limit_min=input.angle_limit_min(),
            angle_limit_max=input.angle_limit_max(),
            angle_decay_factor=input.angle_decay_factor(),
            w_gaussian_min=input.w_gaussian_min(),
            w_gaussian_max=input.w_gaussian_max(),
            gaussian_midpoint=input.gaussian_midpoint(),
            gaussian_steepness=input.gaussian_steepness(),
            low_speed_gaussian_cutoff=input.low_speed_gaussian_cutoff(),
        )

    def _draw_field(ax, x_max=120.0, y_max=53.3, zoom=False, x0=None, pad=10.0):
        ax.set_xlim(0, x_max); ax.set_ylim(0, y_max)
        if zoom and x0 is not None:
            ax.set_xlim(x0 - pad, x0 + pad)
        ax.set_aspect("equal", adjustable="box")
        ax.set_facecolor("#FAFBFD")
        for xv in np.arange(0, x_max + 1, 5):
            ax.axvline(xv, color="#D7DEE8", lw=0.6, alpha=0.6, zorder=0)
        for yv in np.arange(0, y_max + 1, 5):
            ax.axhline(yv, color="#EFF3F8", lw=0.6, alpha=0.6, zorder=0)
        for sp in ax.spines.values():
            sp.set_visible(False)
        ax.tick_params(colors="#8A94A6", labelsize=9)
        ax.set_xlabel("X (yards)", color="#444B59")
        ax.set_ylabel("Y (yards)", color="#444B59")

    def _fallback_Z(pos_xy, pos_off_xy, direction_deg, spd, field_x=120.0, field_y=53.3):
        gx = np.linspace(0, field_x, 241)
        gy = np.linspace(0, field_y, 107)
        X, Y = np.meshgrid(gx, gy)
        x0, y0 = pos_off_xy
        th = np.deg2rad(90 - direction_deg)
        ct, st = np.cos(th), np.sin(th)
        s1 = 2.0 + 0.2 * spd
        s2 = 1.2
        a = (ct**2)/(2*s1**2) + (st**2)/(2*s2**2)
        b = (-2*ct*st)/(2*s1**2) + (2*ct*st)/(2*s2**2)
        c = (st**2)/(2*s1**2) + (ct**2)/(2*s2**2)
        Z = np.exp(-(a*(X-x0)**2 + b*(X-x0)*(Y-y0) + c*(Y-y0)**2))
        return Z, (0, field_x, 0, field_y)

    # --- Play engine: tick based on FPS and gently sweep speed ---
    @reactive.effect
    def _auto_play():
        # Tick according to FPS
        ms = int(1000 / max(1, int(input.fps())))
        reactive.invalidate_later(ms)
        if not bool(input.play()):
            return
        smin = float(input.sweep_min())
        smax = float(input.sweep_max())
        step = float(input.sweep_step())
        cur = float(input.speed())
        # wrap-around sweep
        nxt = cur + step
        if nxt > smax + 1e-9:
            nxt = smin
        ui.update_slider(session, "speed", value=float(round(nxt, 6)))

    @output
    @render.plot
    def contour():
        pos = (float(input.x()), float(input.y()))
        th = np.deg2rad(90 - float(input.dir_deg()))
        spd = float(input.speed())

        if HAVE_MODEL:
            model = PlayerInfluenceModel(**_model_kwargs())
            pos_off = model.compute_offset(pos, float(input.dir_deg()), spd)
            Z = model.base_distribution(
                pos_xy=pos, pos_off_xy=pos_off,
                direction_deg=float(input.dir_deg()),
                speed=spd,
                dist_from_ball=10.0,
            )
            field_x_max = getattr(model, "field_x_max", 120.0)
            field_y_max = getattr(model, "field_y_max", 53.3)
            extent = [0, field_x_max, 0, field_y_max]
        else:
            pos_off = (pos[0] + 1.0 * np.cos(th), pos[1] + 1.0 * np.sin(th))
            Z, (xmin, xmax, ymin, ymax) = _fallback_Z(pos, pos_off, float(input.dir_deg()), spd)
            extent = [xmin, xmax, ymin, ymax]
            field_x_max, field_y_max = xmax, ymax

        fig, ax = plt.subplots(figsize=(10, 7), dpi=140)
        _draw_field(ax, x_max=field_x_max, y_max=field_y_max,
                    zoom=bool(input.zoom()), x0=pos[0], pad=float(input.zoom_pad_x()))
        ax.contourf(
            Z, levels=int(input.levels()), cmap=input.cmap(),
            alpha=float(input.fill_alpha()),
            antialiased=True, origin="lower", extent=extent, zorder=1
        )
        ax.contour(
            Z, levels=int(input.levels()), cmap=input.cmap(),
            alpha=float(input.line_alpha()),
            linewidths=float(input.line_lw()),
            origin="lower", extent=extent, zorder=2
        )
        ax.scatter(*pos, s=22, c="#0B57D0", edgecolors="white", linewidths=1.0, zorder=7)
        ax.scatter(*pos, s=8,  c="white", edgecolors="none", zorder=8)

        # Direction arrow
        dx, dy = 2.2*np.cos(th), 2.2*np.sin(th)
        ax.arrow(pos[0], pos[1], dx, dy, width=0.08, head_width=0.35, head_length=0.70,
                 length_includes_head=True, color="#2F6CFF", alpha=0.9, linewidth=0.6, zorder=9)
        # Orientation arrow
        if bool(input.show_o()):
            tho = np.deg2rad(90 - float(input.o_deg()))
            dxo, dyo = 1.8*np.cos(tho), 1.8*np.sin(tho)
            ax.arrow(pos[0], pos[1], dxo, dyo, width=0.08, head_width=0.32, head_length=0.64,
                     length_includes_head=True, color="#FF8A00", alpha=0.9, linewidth=0.6, zorder=8)

        title = f"{input.name()} – Speed={spd:.2f} yd/s | dir={float(input.dir_deg()):.2f}°"
        if bool(input.show_o()):
            title += f" | o={float(input.o_deg()):.2f}°"
        ax.set_title(title, fontsize=16, pad=10, color="#2B3139")
        return fig

app = App(app_ui, server)

# ---------------- Colab launcher: open in NEW TAB ----------------
try:
    from google.colab import output  # type: ignore

    import shiny

    PORT = random.randint(10000, 20000)

    def _run():
        shiny.run_app(app, host="0.0.0.0", port=PORT, launch_browser=False)

    t = threading.Thread(target=_run, daemon=True)
    t.start()
    time.sleep(1.0)
    # Open the app in a *new browser tab/window*
    output.serve_kernel_port_as_window(PORT)
    print(f"Shiny app is running on proxied port {PORT}. If the tab doesn't load, re-run this cell once.")
except Exception as _e:
    # Not in Colab; print a hint to run locally via: shiny run --reload app.py
    print("Not in Colab. To run locally: `shiny run --reload app.py`")
